In [1]:
import numpy as np
import tnn
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import logging

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
for logger in loggers:
    if "transformers" in logger.name.lower():
        logger.setLevel(logging.ERROR)

In [2]:
dataset = load_dataset("stanfordnlp/imdb")
train = dataset.get("train")
test = dataset.get("test")

In [3]:
num_labels = len(train.unique("label"))
name = "distilbert/distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(name)

In [4]:
def pre_process(examples):
    examples = tokenizer(
        examples["text"],
        max_length=512,
        truncation=True,
        padding="max_length",
    )
    return examples


train_dataset = train.map(
    pre_process,
    batched=True,
    num_proc=2,
)
test_dataset = test.map(
    pre_process,
    batched=True,
    num_proc=2,
)

In [5]:
def collate_fn(batch):
    inputs = {"input_ids": [], "attention_mask": []}
    labels = []
    for example in batch:
        inputs["input_ids"].append(example["input_ids"])
        inputs["attention_mask"].append(example["attention_mask"])
        labels.append(example["label"])

    inputs["input_ids"] = torch.tensor(inputs["input_ids"]).long()
    inputs["attention_mask"] = torch.tensor(inputs["attention_mask"]).long()
    labels = torch.tensor(labels).long()
    return inputs, labels

In [6]:
trainloader = data.DataLoader(
    train_dataset,
    batch_size=32,
    collate_fn=collate_fn,
    drop_last=False,
    shuffle=True,
    pin_memory=True,
    num_workers=2,
)

testloader = data.DataLoader(
    test_dataset,
    batch_size=32,
    collate_fn=collate_fn,
    drop_last=False,
    shuffle=True,
    pin_memory=True,
    num_workers=2,
)


def to(inputs, labels, device, non_blocking):
    inputs = dict(
        map(
            lambda item: (item[0], item[1].to(device, non_blocking=non_blocking)),
            inputs.items(),
        )
    )
    labels = labels.to(device, non_blocking=True)
    return inputs, labels

In [7]:
lr = 5e-5
weight_decay = 1e-4

model = AutoModelForSequenceClassification.from_pretrained(name, num_labels=num_labels)
optim = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
trainer = tnn.Trainer(
    model,
    optim,
    loss_fn,
    trainloader,
    testloader,
    epochs=1,
    store_update_metrics=True,
    unpack_inputs=True,
    device=device,
    to_fn=to,
    path="../training/imdb-distil-bert-sgdm.h5",
    verbose=True,
    profile=True,
)

In [9]:
sgdm_metrics = trainer.train()

model using cuda
training started
(epoch: 1/10): (train loss: 0.6917, test loss: 0.6821, train acc: 52.32%, test acc: 63.88%)
(gpu memory profile): (average allocated: 811.0 MB, average reserved: 5989.0 MB)
(duration info): (epoch duration: 0:03:51, elapsed time: 0:03:51)
(epoch: 2/10): (train loss: 0.6589, test loss: 0.5759, train acc: 68.68%, test acc: 83.96%)
(gpu memory profile): (average allocated: 811.0 MB, average reserved: 5990.0 MB)
(duration info): (epoch duration: 0:03:49, elapsed time: 0:07:40)
(epoch: 3/10): (train loss: 0.3986, test loss: 0.2895, train acc: 85.66%, test acc: 88.26%)
(gpu memory profile): (average allocated: 811.0 MB, average reserved: 5990.0 MB)
(duration info): (epoch duration: 0:03:49, elapsed time: 0:11:30)
(epoch: 4/10): (train loss: 0.2823, test loss: 0.2591, train acc: 88.87%, test acc: 89.31%)
(gpu memory profile): (average allocated: 811.0 MB, average reserved: 5990.0 MB)
(duration info): (epoch duration: 0:03:49, elapsed time: 0:15:20)
(epoch: 5/